*Role Title and Role Code are similar based on pandas
*Instead of using hashing, can consider cantor(m,n) = 1/2 * (m+n)(m+n+1)+m. Beware of hash collision
*PCA / SVD are more suited for continuous values. For discrete variables, consider using MCA
1. Feature Extraction - Using Genetic Algorithm / Greedy Algorithm [sklearn.feature_selection.RFE, RFECV , Gradient Boosting
2. Try using Stacked Classifier 
  * np.mean
  * weighted mean
  * jaccard score
  * pass the results from each classifer into another RF / Linear Regression
3. Data Exploratory - Find if train has more classes compared to test ? 
4. Try to run on Spark
5. Stratified K Fold Cross Validation
6. Use GridSearchCV to optimize Logistic Classifier
7. Consider having equal +ve and -ve examples in each train dataset

8. Count Analysis (Use Log Odd Ratio) + Two Class AdaBoosted Decision Tree
9. Compare between average and voting for Ensemble. How to combine Logistic (X,y) and RFC (X1,y1)
10. Make a graph of count / np.log rf_data[] against y [scatterplot]

* ACTION is 1 if the resource was approved, 0 if the resource was not
* RESOURCE - An ID for each resource
* MGR_ID- The EMPLOYEE ID of the manager of the current EMPLOYEE ID record; an employee may have only one manager at a time
* ROLE_ROLLUP_1 - Company role grouping category id 1 (e.g. US Engineering)
* ROLE_ROLLUP_2 - Company role grouping category id 2 (e.g. US Retail)
* ROLE_DEPTNAME - Company role department description (e.g. Retail)
* ROLE_TITLE - Company role business title description (e.g. Senior Engineering Retail Manager) - Same as Role Code
* ROLE_FAMILY_DESC - Company role family extended description (e.g. Retail Manager, Software Engineering)
* ROLE_FAMILY - Company role family description (e.g. Retail Manager)
* ROLE_CODE - Company role code; this code is unique to each role (e.g. Manager)


In [1]:
#inside a ipython/nb session
# Original Data in Pandas Dataframe : data
# Sparse Array for Training Data : X
# Sparse Array for Test Data : X_test
# Label for X only : y
# Offset : test_offset
# rf_data : Data Set for Random Forest Tree


#inside a ipython/nb session
# %store encoder
# Original Data in Pandas Dataframe
# %store data
# Sparse Array for Training Data
# %store X
# Sparse Array for Test Data
# %store X_test
# Label for X only
# %store y
# %store features
# %store header
# Offset 
# %store test_offset
# Data Set for Random Forest Tree
# %store rf_data

%store -r encoder
%store -r data
%store -r X
%store -r X_test
%store -r y
%store -r features
%store -r header
%store -r test_offset
%store -r rf_data


# fileObject = open("amazon.pickle",'wb') 
# pickle.dump(encoder,fileObject)   
# pickle.dump(data,fileObject)   
# pickle.dump(X,fileObject)   
# pickle.dump(X_test,fileObject)   
# pickle.dump(y,fileObject)   
# pickle.dump(features,fileObject)   
# pickle.dump(header,fileObject)   
# pickle.dump(test_offset,fileObject)   
# pickle.dump(rf_data,fileObject)   
# fileObject.close()

In [2]:
import pandas as pd
from pivottablejs import pivot_ui
import seaborn as sb
import matplotlib.pyplot as plt  
from ipywidgets import widgets
import numpy as np
from sklearn import (metrics, cross_validation, linear_model, preprocessing)
from sklearn.metrics import log_loss
from scipy.optimize import minimize
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import (GradientBoostingClassifier,RandomForestClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn import grid_search
from itertools import combinations
import pickle

%matplotlib inline

sb.set(style="ticks", color_codes=True)
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')
#g = sb.pairplot(df)

In [ ]:
#len(df['RESOURCE'].value_counts())
#df['RESOURCE'].plot.hist(bins=7518)
#df.head()
#dfc = df.iloc[:,1:10] 
dfc = df.iloc[:,:]
for col in df.columns:
  dfc[col] = df[col].astype('category')
dfc.describe()


In [ ]:
total_one_hot = 0
for col in dfc.columns[1:]:
  total_one_hot += len(pd.unique(dfc[col]))
print("Total Features :",total_one_hot)

dfc['ACTION'].value_counts(normalize=True)

In [ ]:
#Observation Only
#test = pd.pivot_table(dfc,index=['ROLE_TITLE','ROLE_CODE'],aggfunc='count',dropna=True)
#dfc.pivot_table(index='ROLE_TITLE',aggfunc=lambda x:len(x.unique()))
mult = {}
for c in df.columns:
    temp = df.pivot_table(index=c,aggfunc=lambda x:len(x.unique()),fill_value=0).apply(np.max)
    mult[c] = temp

multpd = pd.DataFrame(mult)

multpd[multpd.columns[1:]]
#Role Code to Role Title => 1:1
#Role Family to Role Code => 1 : M
#Role Family to Role Title => 1 : M
#Therefore Role Code = Role Title 

In [ ]:
# Action, Resource, Role_Code, Role_DeptName, Role_Family, Role_Family_Desc, Role_Roll_1, Role_Roll_2, Role_Title
mgr = df.pivot_table(index='MGR_ID',aggfunc= lambda x:len(x.unique()),fill_value=0)
mgr.hist(bins=20,figsize=(10,10))


In [ ]:
# Data Frame Analysis (Obeservation Only)
# # pd.set_option('display.max_rows',500)
# mgr_id_high = mgr.sort_values(by=['RESOURCE'],ascending=False)['RESOURCE'] > 30
# print("MGR where resources less:", len(mgr_id_high.index.values[~mgr_id_high.values]))
# print("MGR where resources more:", len(mgr_id_high.index.values[mgr_id_high.values]))
# print("Total:",len(k.index.values))
# #df[df['MGR_ID'].isin(k)]

# #df[(df['ACTION']==0)][['ACTION']].count()
# mgr_total = df[(df['MGR_ID'].isin(mgr_id_high.index.values[k.values]))][['ACTION']].count()
# mgr_action_1 = df[(df['MGR_ID'].isin(mgr_id_high.index.values[k.values])) & (df['ACTION']==1)][['ACTION']].count()
# mgr_action_0 = df[(df['MGR_ID'].isin(mgr_id_high.index.values[k.values])) & (df['ACTION']==0)][['ACTION']].count()
# print("Percentage MGR : ",mgr_action_0/mgr_total)
# #df[['ACTION','RESOURCE','MGR_ID']]
# # k.pivot_table(index='RESOURCE',aggfunc='count')

# mgr_total_o = df[(~df['MGR_ID'].isin(mgr_id_high.index.values[k.values]))][['ACTION']].count()
# mgr_action_1_o = df[(~df['MGR_ID'].isin(mgr_id_high.index.values[k.values])) & (df['ACTION']==1)][['ACTION']].count()
# mgr_action_0_o = df[(~df['MGR_ID'].isin(mgr_id_high.index.values[k.values])) & (df['ACTION']==0)][['ACTION']].count()
# print("Percentage MGR : ",mgr_action_0_o/mgr_total_o)
# print("Mgr_less", mgr_total_o,"   ",mgr_action_1_o,"    ",mgr_action_0_o)
# print("Mgr_more", mgr_total,"   ",mgr_action_1,"    ",mgr_action_0)

In [ ]:
# Action, Resource, Role_Code, Role_DeptName, Role_Family, Role_Family_Desc, Role_Roll_1, Role_Roll_2, Role_Title
rollup1 = df.pivot_table(index='ROLE_ROLLUP_1',aggfunc= lambda x:len(x.unique()),fill_value=0)
rollup1['ROLE_ROLLUP_2'].hist()
rollup1['ROLE_ROLLUP_2'].value_counts(normalize=True)

In [ ]:
rollup2 = df.pivot_table(index='ROLE_ROLLUP_2',aggfunc= lambda x:len(x.unique()),fill_value=0)
rollup2['ROLE_ROLLUP_1'].hist()
rollup2['ROLE_ROLLUP_1'].value_counts(normalize=True)

In [ ]:
abc = pd.concat([df['ACTION'],rf_data[:test_offset][rf_data.columns.values[6:12]]], axis=1)
sb.regplot(abc['cntROLE_ROLLUP_1&ROLE_ROLLUP_2'],abc['ACTION'],scatter=True)
# abc[['ACTION','cntMGR_ID']]

In [15]:
def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))


In [ ]:
def cantor(pairs):
    result2 = 0.5 * (pairs[0] + pairs[1]) * (pairs[0] + pairs[1] +1) + pairs[1]
    if(len(pairs) == 2): 
        return result2
    if(len(pairs) == 3 ):
        result3 = 0.5 * (result2 + pairs[2]) * (result2 + pairs[2] +1) + pairs[2]
        return result3
    

In [ ]:
# Convert from dataframe to np.array
y = np.ravel(df.iloc[:,:1])
X = np.array(df.iloc[:,1:9])
X_test = np.array(dft.iloc[:,1:9])
header = list(df.columns[1:])

data = np.vstack((X,X_test))

In [ ]:
# yc = combinations(range(n),2)
# to view all the combinations
# list(yc) 

#train data[:32769,:]
#test data[32769:,:]
#tuple is used here so that the result is hashable

test_offset = 32769
# Get Combinations of 2 features 
for indices in combinations(range(8),2):
    #data = np.column_stack((data,list(ctypes.c_size_t(hash(tuple(v))).value for v in data[:,indices]))) 
    #data = np.column_stack((data,list(np.uint64(hash(tuple(v))) for v in data[:,indices]))) 
    #data = np.column_stack((data,list(hash(tuple(v)) for v in data[:,indices]))) 
    data = np.column_stack((data,list(cantor(tuple(v)) for v in data[:,indices]))) 
    a,b = indices
    header.append(header[a] +"&"+ header[b])
print('2 Combinations')
print(data.shape)
print(np.any(data < 0))
print(np.ndarray.min(data))
print(np.ndarray.max(data))
    
# Get Combinations of 3 features 
for indices in combinations(range(8),3):
    data = np.column_stack((data,list(cantor(tuple(v)) for v in data[:,indices]))) 
    a,b,c = indices
    header.append(header[a] +"&"+ header[b]+ "&" + header[c])

    
#Check loaded correctly with non-zero elements    
print('3 Combinations')
print(data.shape)
print(np.any(data < 0))
print(np.ndarray.min(data))
print(np.ndarray.max(data))

In [ ]:
# Simplify the encoding. Probably not essential. Could possibly improve performance
# Vertical Stacking is required so that all the features are captured. There might be some features in test set not in train set

le = preprocessing.LabelEncoder()
data_e = []
# To perform column wise operations, use Transpose function
for i in data.T:
    le.fit(i)
    data_e.append(le.transform(i))

data = np.asarray(data_e).T

print(np.any(data < 0))
print(np.ndarray.min(data))
print(np.ndarray.max(data))



In [ ]:
# #Code to help Visualize what the algorithm is doing 
# t_data = []
# test_data = np.array([[1,2,3,4],[1,2,2,0],[1,2,3,4],[1,4,5,6],[2,3,4,5]])
# print(test_data.shape)

# #if the combination of categories are the same, they will be hashed to the same value. Eg 1 row with (0,1,0) will have same 
# #hash value of another row with (0,1,0)
# for indices in combinations(range(4),2):
#     t_data.append(hash(tuple(v)) for v in test_data[:,indices])
#     #print(v for v in test_data[:,indices])
#     print(indices)
    
# #Comparison 1 - Combinations are (0,1),(0,2),(0,3),(1,2),(1,3),(2,3)
# # for v in test_data[:,(0,1)]:
# #   print(hash(tuple(v)))

# #Comparison 2 - t_data[0] corresponds to (0,1)
# #tuple(t_data[0])



In [ ]:
#Initial Encoding for all features

encoder = preprocessing.OneHotEncoder(dtype=np.uint32)
# we want to encode the category IDs encountered both in
# the training and the test set, so we fit the encoder on both
encoder.fit(data)
X = encoder.transform(data[:test_offset,:])
X_test = encoder.transform(data[test_offset:,:])
print(X.shape)
print(X_test.shape)

print("Total One Hot Features",encoder.active_features_)
print("Boundaries of each Feature",len(encoder.feature_indices_),encoder.feature_indices_)
print("Number of unique values in each feature",len(encoder.n_values_),encoder.n_values_)
# print(np.unique(data[:,0]))
# print(np.unique(data[:,1]))
# sb.barplot(x=np.arange(92),y=encoder.n_values_)

In [ ]:
#Feature Selection - Gradient Boosting (Filtering)
dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(X,y)
print(dtc.feature_importances_)
features = dtc.feature_importances_

#sb.distplot(feat,bins=100)
#print(np.argmax(features),np.max(features))
# Sum features in the same bin as defined by encoder.n_values_
x = 0
start = 0
prob_sum = []
#prob_ref = dict()
for i in encoder.n_values_ : 
    prob = features[start:i].sum()
    prob_sum.append(prob)
    #prob_ref[header[x]] = prob 
    start = i + 1
    #x += 1

In [ ]:
# Stability Selection using Randomized Lasso

rlasso = linear_model.RandomizedLogisticRegression(C=1)
rlasso.fit(X,y)

rlasso_score = rlasso.scores_

x = 0
start = 0
prob_sum = []
for i in encoder.n_values_ : 
    prob_sum.append(rlasso_score[start:i].sum())
    start = i + 1

print(rlasso_score)
print(prob_sum)


In [ ]:
#Feature Selection - Greedy Algorithm (Wrapping)

estimator = linear_model.LogisticRegression(C=1)
selector = RFE(estimator)
selector = selector.fit(X, y)
selector.ranking_

In [ ]:
# Pick out the most useful features

# sb.barplot(y=prob_sum,x=np.arange(92))
#hsb = sb.hist(prob_sum,xlim=0.1)
#Look at bar plot to see what threshold of the features to select
# ufeatures = []
# for i,j in enumerate(prob_sum):
#     if (j>0) : ufeatures.append(i)
# print(ufeatures)
[header[i] for i in ufeatures]

In [ ]:
# Kaggle Greedy Algorithm result
#ufeatures = [0, 8, 9, 10, 19, 34, 36, 37, 38, 41, 42, 43, 47, 53, 55, 60, 61, 63, 64, 67, 69, 71, 75, 81, 82, 85]
ufeatures = [0, 1, 7, 8, 9, 10, 36, 37, 38, 41, 42, 43, 47, 51, 53, 56, 60, 61, 63, 64, 66, 67, 69, 71, 75, 79, 85, 91]
print(ufeatures)

In [ ]:
#OneHotEncoder for the best features only

encode_bfeat = preprocessing.OneHotEncoder(dtype=np.uint32)
# we want to encode the category IDs encountered both in
# the training and the test set, so we fit the encoder on both
encode_bfeat.fit(data[:,ufeatures])
X = encode_bfeat.transform(data[:test_offset,ufeatures])
X_test = encode_bfeat.transform(data[test_offset:,ufeatures])
print(X.shape)
print(X_test.shape)

In [ ]:
# Testing Random Forest Tree
rf_head = [0,1,22,4,6,35]
rf_data = pd.DataFrame(data)
rf_data = rf_data[rf_head]
rf_data.columns = [header[i] for i in rf_head]

In [ ]:
#https://www.kaggle.com/c/caterpillar-tube-pricing/forums/t/15748/strategies-to-encode-categorical-variables-with-many-categories
# One Time Run Completed
#Count/Freq
# for col in rf_data.columns:
#     rf_data['cnt'+col] = 0
#     groups = rf_data.groupby([col])
#     # gid is the grouped col id, group is the actionable pandas data structure
#     for gid, group in groups:
#         #Count total number of rows with the same (MGR_ID,RESOURCE_ID etc) in the entire dataset
#         count = group[col].count()
#         #For all rows with the same "MGR_ID"/"RESOURCE_ID" , put the same count calculated in the previous step
#         rf_data['cnt'+col].ix[group.index] = count 
#     #rf_data['cnt'+col] = rf_data['cnt'+col].apply(np.log) # could check if this is neccesary, I think probably not
    
# # Percent of dept that is this resource
# for col in rf_data.columns[1:6]:
#     rf_data['Duse'+col] = 0.0
#     groups = rf_data.groupby([col])
#     for name, group in groups:
#         grps = group.groupby(['RESOURCE'])
#         for rsrc, grp in grps:
#             rf_data['Duse'+col].ix[grp.index] = float(len(grp.index)) / float(len(group.index) )

# #Number of resources that a manager manages
# for col in rf_data.columns[0:1]:
#     rf_data['Mdeps'+col] = 0
#     groups = rf_data.groupby(['MGR_ID'])
#     for name, group in groups:
#         rf_data['Mdeps'+col].ix[group.index] = len(group[col].unique())

X = rf_data.values[:test_offset,6:18]
X_test = rf_data.values[test_offset:,6:18]

# X = rf_data.values[:test_offset,:]
# X_test = rf_data.values[test_offset:,:]

In [ ]:
#Logistic Regression

SEED = 42
mean_auc = 0.0

n = 10  # repeat the CV procedure 10 times to get more precise results
model = linear_model.LogisticRegression(C=1,penalty='l2',class_weight='balanced')
#model = BernoulliNB(alpha=0.005)
#model = ensemble.ExtraTreesClassifier(criterion='entropy',n_jobs=-1,verbose=1)
for i in range(n):
    # for each iteration, randomly hold out 20% of the data as CV set
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(X, y, test_size=.20, random_state=i*SEED)

    # if you want to perform feature selection / hyperparameter
    # optimization, this is where you want to do it

    # train model and make predictions
    model.fit(X_train, y_train) 
    preds = model.predict_proba(X_cv)[:, 1]

    # compute AUC metric for this CV fold
    fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
    roc_auc = metrics.auc(fpr, tpr)
    print("AUC (fold %d/%d): %f" % (i + 1, n, roc_auc))
    mean_auc += roc_auc
    # To print curve
#     plt.plot(fpr,tpr)
#     plt.show() 

print("Mean AUC: %f" % (mean_auc/n))

# === Predictions === #
# When making predictions, retrain the model on the whole training set
model.fit(X, y)
preds = model.predict_proba(X_test)[:, 1]
save_results(preds, "submit_log.csv")

In [ ]:
# Numpy Operations
# print(np.fromiter(t_data[0],np.int32))
# print(np.fromiter(t_data[1],np.int32))
# print(np.fromiter(t_data[2],np.int32))
# print(np.fromiter(t_data[3],np.int32))
# print(np.fromiter(t_data[4],np.int32))
# print(np.fromiter(t_data[5],np.int32))

# Convert to numpy array - Use np.array(list)
# for i in nparray - iterate through rows
# for i in nparray.T - iterate through columns
# Initialize np.zeros(5)

# To view generator operators (Use list,tuples)
# for i,j in enumerate(t_data):
#     #lst_array[i] = np.array(list(j))
#     lst_array[i] = np.fromiter(j,np.float)

In [ ]:
#GridSearchCV / Pipeline

SEED = 35

clf1 = RandomForestClassifier(max_depth=None,max_features='sqrt',min_samples_split=9,random_state=SEED)
clf2 = ensemble.ExtraTreesClassifier(max_features='sqrt',min_samples_split=8,max_depth=None,random_state=SEED)
clf3 = GradientBoostingClassifier(max_depth=20,min_samples_split=9,random_state=SEED,learning_rate=0.2,n_estimators=50)
clf4 = linear_model.LogisticRegression(C=1,penalty='l2',class_weight='balanced')

#estimators = [('mrf',RandomForestClassifier()),('etc',ExtraTreesClassifier()),('gbc',GradientBoostingClassifier())]
#pipeline = Pipeline([('mrf',RandomForestClassifier())])

eclf = ensemble.VotingClassifier(estimators=[('rfc',clf1),('etc',clf2),('gbc',clf3),('lr',clf4)],voting='soft')
params = {
    'rfc__n_estimators' : [50,100,500],
    'etc__n_estimators' : [50,100,500],
    'gbc__max_depth' : [5,20]
}

grid = grid_search.GridSearchCV(estimator=eclf,param_grid=params,scoring='roc_auc',n_jobs=-1)
grid.fit(X,y)

print(grid.best_score_)
print(grid.best_estimator_)
pred_grid = grid.predict_proba(X_test)
print(pred_grid[:, 1])

save_results(pred_grid[:, 1], "submit_v1.csv")


In [ ]:
# Scenario 1 : Will using the whole sparse array outperform using the normal
# ET rf_data [ 0.86885664  0.859536    0.86248539]
# ET Full Set [ 0.85228669  0.83864796  0.84169196]

# tclf1 = ensemble.ExtraTreesClassifier(max_features='sqrt',max_depth=None,min_samples_split=9,n_estimators=50)
# etsrf = cross_validation.cross_val_score(clf2,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)
# etsf = cross_validation.cross_val_score(tclf1,X,y,scoring='roc_auc',cv=3,n_jobs=-1)
# print("ET on Count Values",etsrf)
# print("ET Full Set",etsf)

# Scenario 2 : Performance with AdaBoost (Random Forest)
# tclf1 = DecisionTreeClassifier(max_features='sqrt',max_depth=None)
# tclf2 = RandomForestClassifier(max_depth=None,max_features='sqrt')
# aclf1 = ensemble.AdaBoostClassifier(base_estimator=tclf1,n_estimators=500,learning_rate=1)
# aclf2 = ensemble.AdaBoostClassifier(base_estimator=tclf2,n_estimators=500,learning_rate=1)
# adadtc = cross_validation.cross_val_score(aclf1,X,y,scoring='roc_auc',cv=3,n_jobs=-1)
# adarff = cross_validation.cross_val_score(aclf2,X,y,scoring='roc_auc',cv=3,n_jobs=-1)
# adarfp = cross_validation.cross_val_score(aclf2,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)
# print("AdaBoost on Decision Tree",adadtc)
# print("AdaBoost on Random Forest on Full Dataset",adarff)
# print("AdaBoost on Random Forest on Count Values",adarfp)

# Scenario 3 : Calibrated Random Forest vs uncalibrated
# clf1_calib = CalibrationClassifierCV(base_estimator=clf1, cv=3, method='isotonic')
# rfc_calib.train(Xf,y)
# rfc = cross_validation.cross_val_score(clf1,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)
# rfcc = cross_validation.cross_val_score(rfcc,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)

# Scenario 5 : Ensemble with Logistic Regression 
# rs = cross_validation.ShuffleSplit(len(y),n_iter=1,test_size=.5, random_state=SEED)
# for set1, set2 in rs:
#     # Fit Set 1 to Set 2
#     clf1.fit(Xf[set1],y[set1])
#     clf2.fit(Xf[set1],y[set1])
#     clf3.fit(Xf[set1],y[set1])
#     clf4.fit(X[set1],y[set1])
    
#     pclf1 = clf1.predict_proba(Xf[set1])[:, 1]
#     pclf2 = clf1.predict_proba(Xf[set1])[:, 1]
#     pclf3 = clf1.predict_proba(Xf[set1])[:, 1]
#     pclf4 = clf1.predict_proba(X[set1])[:, 1]
    
#     sclf1 = metrics.roc_auc_score(y[set2], pclf1)
#     sclf2 = metrics.roc_auc_score(y[set2], pclf2)
#     sclf3 = metrics.roc_auc_score(y[set2], pclf3)
#     sclf4 = metrics.roc_auc_score(y[set2], pclf4)

#     preds = np.hstack((pclf1,pclf2,pclf3,pclf4)).reshape(4,len(pclf1)).transpose()
# #     preds[preds>0.9999999]=0.9999999
# #     preds[preds<0.0000001]=0.0000001
# #     preds = -np.log((1-preds)/preds)
#     modelEN1 = linear_model.LogisticRegression()
#     modelEN1.fit(preds, y[set2])
#     print modelEN1.coef_
    
#     # Fit Set 2 to Set 1
#     clf1.fit(Xf[set2],y[set2])
#     clf2.fit(Xf[set2],y[set2])
#     clf3.fit(Xf[set2],y[set2])
#     clf4.fit(X[set2],y[set2])
    
#     pclf1 = clf1.predict_proba(Xf[set2])[:, 1]
#     pclf2 = clf1.predict_proba(Xf[set2])[:, 1]
#     pclf3 = clf1.predict_proba(Xf[set2])[:, 1]
#     pclf4 = clf1.predict_proba(X[set2])[:, 1]
    
#     sclf1 = metrics.roc_auc_score(y[set1], pclf1)
#     sclf2 = metrics.roc_auc_score(y[set1], pclf2)
#     sclf3 = metrics.roc_auc_score(y[set1], pclf3)
#     sclf4 = metrics.roc_auc_score(y[set1], pclf4)

#     preds = np.hstack((pclf1,pclf2,pclf3,pclf4)).reshape(4,len(pclf1)).transpose()
# #     preds[preds>0.9999999]=0.9999999
# #     preds[preds<0.0000001]=0.0000001
# #     preds = -np.log((1-preds)/preds)
#     modelEN2 = linear_model.LogisticRegression()
#     modelEN2.fit(preds, y[set1])
#     print modelEN2.coef_

#     coefRF = modelEN1.coef_[0][0] + modelEN2.coef_[0][0]
#     coefXT = modelEN1.coef_[0][1] + modelEN2.coef_[0][1]
#     coefGB = modelEN1.coef_[0][2] + modelEN2.coef_[0][2]
#     coefLR = modelEN1.coef_[0][3] + modelEN2.coef_[0][3]

# # Cross Validation
# rscv = cross_validation.ShuffleSplit(len(y),n_iter=1,test_size=.5, random_state=SEED*2)
# for set1, set2 in rscv:
#     #Set 1
#     clf1.fit(Xf[set1],y[set1])
#     clf2.fit(Xf[set1],y[set1])
#     clf3.fit(Xf[set1],y[set1])
#     clf4.fit(X[set1],y[set1])
    
#     pclf1 = clf1.predict_proba(Xf[set2])[:, 1]
#     pclf2 = clf1.predict_proba(Xf[set2])[:, 1]
#     pclf3 = clf1.predict_proba(Xf[set2])[:, 1]
#     pclf4 = clf1.predict_proba(X[set2])[:, 1]
#     pclfe = coefRF * pclf1 + coefXT * pclf2 + coefGB * pclf3 + coefLR * pclf4
    
#     sclf1 = metrics.roc_auc_score(y[set2], pclf1)
#     sclf2 = metrics.roc_auc_score(y[set2], pclf2)
#     sclf3 = metrics.roc_auc_score(y[set2], pclf3)
#     sclf4 = metrics.roc_auc_score(y[set2], pclf4)
#     sclfe = metrics.roc_auc_score(y[set2], pclfe)
#     #Set 2
#     clf1.fit(Xf[set2],y[set2])
#     clf2.fit(Xf[set2],y[set2])
#     clf3.fit(Xf[set2],y[set2])
#     clf4.fit(X[set2],y[set2])
    
#     pclf1 = clf1.predict_proba(Xf[set1])[:, 1]
#     pclf2 = clf1.predict_proba(Xf[set1])[:, 1]
#     pclf3 = clf1.predict_proba(Xf[set1])[:, 1]
#     pclf4 = clf1.predict_proba(X[set1])[:, 1]
#     pclfe = coefRF * pclf1 + coefXT * pclf2 + coefGB * pclf3 + coefLR * pclf4
    
#     sclf1 = metrics.roc_auc_score(y[set1], pclf1)
#     sclf2 = metrics.roc_auc_score(y[set1], pclf2)
#     sclf3 = metrics.roc_auc_score(y[set1], pclf3)
#     sclf4 = metrics.roc_auc_score(y[set1], pclf4)
#     sclfe = metrics.roc_auc_score(y[set1], pclfe)

    

In [ ]:
#Ensemble

Xf = rf_data.values[:test_offset,6:18]
Xf_test = rf_data.values[test_offset:,6:18]

SEED = 42

clf1 = RandomForestClassifier(criterion='entropy',n_estimators=500,max_depth=None,max_features='sqrt',min_samples_split=9,random_state=SEED)
clf2 = ensemble.ExtraTreesClassifier(criterion='entropy',n_estimators=500,max_features='sqrt',min_samples_split=8,max_depth=None,random_state=SEED)
clf3 = GradientBoostingClassifier(max_depth=20,min_samples_split=9,random_state=SEED,learning_rate=0.2,n_estimators=50)
clf4 = linear_model.LogisticRegression(C=1,penalty='l2',class_weight='balanced')

## Does not support sparse array
# rfs = cross_validation.cross_val_score(clf1,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)
## Supports Sparse array
# ets = cross_validation.cross_val_score(clf2,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)
# gbs = cross_validation.cross_val_score(clf3,Xf,y,scoring='roc_auc',cv=3,n_jobs=-1)
# lrs = cross_validation.cross_val_score(clf4,X,y,scoring='roc_auc',cv=3,n_jobs=-1)
# print("RF",rfs)
# print("ET",ets)
# print("GB",gbs)
# print("LR",lrs)


# Scenario 4 : Averaging (Find Optimal Weights) // Incomplete!
# Use Log Loss with Scipy.Optimize to find the suitable weights

pave_coll = []
pave_y = []
rs = cross_validation.ShuffleSplit(len(y),n_iter=3,test_size=.2, random_state=SEED)
for train, test in rs:
    clf1.fit(Xf[train],y[train])
    clf2.fit(Xf[train],y[train])
    clf3.fit(Xf[train],y[train])
    clf4.fit(X[train],y[train])
    
    pclf1 = clf1.predict_proba(Xf[test])[:, 1]
    pclf2 = clf2.predict_proba(Xf[test])[:, 1]
    pclf3 = clf3.predict_proba(Xf[test])[:, 1]
    pclf4 = clf4.predict_proba(X[test])[:, 1]

    pave = np.vstack((pclf1,pclf2,pclf3,pclf4)).transpose()
    pave_coll.append(pave)
    pave_y.append(y[test]) 
    
pcoll = np.vstack(pave_coll)
py = np.hstack(pave_y)

# Optimize
# https://www.kaggle.com/hsperr/otto-group-product-classification-challenge/finding-ensamble-weights
def log_loss_func(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    prediction = np.asarray(np.mat(pcoll)*np.mat(weights.reshape(4,1)))
    return metrics.roc_auc_score(py, prediction)
    
starting_values = np.array([0.25,0.25,0.25,0.25])
# Equality constraint means that the constraint function result is to be zero whereas inequality means that it is to be non-negative
cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
#our weights are bound between 0 and 1
bounds = [(0,1)]*4
res = minimize(log_loss_func, starting_values, method='SLSQP', bounds=bounds, constraints=cons)
print('Ensamble Score: {best_score}'.format(best_score=res['fun']))
print('Best Weights: {weights}'.format(weights=res['x']))
    
# === Predictions === #

clf1.fit(Xf, y)
clf2.fit(Xf, y)
clf3.fit(Xf, y)
clf4.fit(X, y)

predsRF = clf1.predict_proba(Xf_test)[:, 1]
predsET = clf2.predict_proba(Xf_test)[:, 1]
predsGB = clf3.predict_proba(Xf_test)[:, 1]
predsLR = clf4.predict_proba(X_test)[:, 1]

# Scenario 4

predsO = np.vstack((predsRF,predsET,predsGB,predsLR)).transpose()
predsOF = np.asarray(np.mat(predsO) * np.mat(res['x'].reshape(4,1)))
save_results(predsOF, "submit_optimize.csv") 

# predsC = (predsRF + predsET + predsGB + 2*predsLR)/5
# save_results(predsC, "submit_ave.csv") #0.91042 - (predsRF + predsET + predsGB + 2*predsLR)/5
# predsS = (np.amax(np.vstack((predsLR,predsGB,predsRF,predsET)),axis=0))
# save_results(predsS, "submit_vote.csv") #0.86148

# Scenario 5
# predsE = coefRF * predsRF + coefXT * predsXT + coefGB * predsGB + coefLR * predsLR
# save_results(predsE, "submit_ensemble.csv") 




array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [8]:
# sb.distplot(pcoll[:,1])
# sb.distplot(pclf2)
tx = np.array([[0,1,2,3],[4,5,6,7],[8,9,10,11]])
# y = np.hstack((tx))
# y
weight = np.array([[0.25],[0.25],[0.25],[0.25]])
# result = np.asarray(np.mat(tx) * np.mat(weight))
weight.shape

w = [0.25,0.25,0.25,0.25]
rw = np.asarray(w).reshape((4,1))
rw

array([[ 0.25],
       [ 0.25],
       [ 0.25],
       [ 0.25]])

In [ ]:
predsS = (np.amax(np.vstack((predsLR,predsGB,predsRF,predsET)),axis=0))
save_results(predsS, "submit_v2.csv") 

AUC = Logistic Regression returns the probability that a certain input features would be in a certain class
ROC curve visualizaes all possible thresholds
Misclassification rate is error rate for a single threshold


Attempt with original features only : CV : ... , Public : 0.88515, Private : 0.88205

Attempt with original features + combination of 3 : CV : 0.886375 , Public : 0.90141, Private 0.89582 
ExtraTreeClassifier = submit_etc.csv


Attempt with original features + combination of 3 : CV : 0.886375 , Public : 0.90141, Private 0.89582 
Logistic C=1, class_weight='balanced',penalty='l2' = submit_c3.csv

Attempt with DTC selected features : CV :  0.886555 (threshold >0) , Public : , Private 
j>0.001 or 0 
Logistic

Attempt with Random Lasso  : CV : 0.886439   , Public : , Private 
j>0
Logistic C=1, class_weight='balanced',penalty='l2' = submit_rl.csv

Cheat Sheet from Kaggle, CV : 0.895952
Logistic C=1, class_weight='balanced',penalty='l2' = submit_kc.csv - Private = 0.89872

In [ ]:
X_test.shape

In [ ]:
Xf

In [ ]:
te1 = np.array([1,2,30,4,5])
te2 = np.array([11,7,8,9,10])
te3 = np.array([3,12,13,14,15])
teh = np.vstack((te1,te2,te3))
print(teh)
print(np.amax(teh,axis=0))

In [ ]:
teh = np.hstack((te1,te2,te3)).reshape(3,5).transpose()
print(teh)

In [ ]:
fileObject = open("test.pickle",'wb') 
pickle.dump(RandomForestClassifier,fileObject)   
fileObject.close()

In [ ]:
who_ls
%reset_selective -f b